In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
from sklearn.preprocessing import OneHotEncoder

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Preprocessing of data
data = data.drop(['CustomerId','RowNumber','Surname'],axis=1)

In [5]:
#Encode categorical value
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [6]:
#One hot encoding of geography
onehot_encoder_geo=OneHotEncoder()
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [7]:
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
# combine the encoded column with original data
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
#Split the data into features and target
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [11]:
# Split the train and test data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
# Scaler the feature
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [13]:
#Save the encoder and scaler for later use
with open('label_encoder_gender1.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo1.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler1.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Reegression Problem Statement


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [23]:
X_train.shape[1]

12

In [24]:
# Build the model
model = Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

#Set up the tensor board
log_dir='regression/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [36]:
#set up for the Early stop
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [37]:
#Train the model
history= model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 49480.4375 - mae: 49480.4375 - val_loss: 50266.3945 - val_mae: 50266.3945
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 49471.2812 - mae: 49471.2812 - val_loss: 50273.5938 - val_mae: 50273.5938
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 49466.7227 - mae: 49466.7227 - val_loss: 50264.0781 - val_mae: 50264.0781
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 49458.6602 - mae: 49458.6602 - val_loss: 50263.4414 - val_mae: 50263.4414
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 49453.4922 - mae: 49453.4922 - val_loss: 50266.0508 - val_mae: 50266.0508
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 49445.2227 - mae: 49445.2227 - val_loss: 50263.6602 - val_mae: 50263.6602
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 49439.1211 - mae: 49439.1211 - 

In [39]:
%load_ext tensorboard

In [44]:
%tensorboard --logdir regression/fit

Reusing TensorBoard on port 6007 (pid 18188), started 0:00:07 ago. (Use '!kill 18188' to kill it.)

In [45]:
# Evaluate the model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 0s 1ms/step - loss: 50263.1758 - mae: 50263.1758
Test MAE: 50263.17578125


In [46]:
model.save('regression_model.h5')

c:\Users\Pradum Gupta\OneDrive\Desktop\udemy\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
